In [1]:
import warnings

warnings.filterwarnings('ignore')

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

pd.options.display.max_columns = 999
RANDOM_STATE = 42

DATA_PATH = 'data/final_train.csv'
MODEL_AUTOML_PATH = 'results/automl-single-model-2021-28-09-21-28-03/model.joblib'
MODEL_DECISION_TREE_PATH = 'results/prunned_tree/prunned_tree.joblib'
MODEL_DECISION_TREE_3_CAT_PATH = 'results/3categories/3categories_tree.joblib'

In [2]:
def summary(model, x, y):
    y_hat = model.predict(x)
    print('-'*40)
    print('Raport klasyfikacyjny')
    print()
    print(classification_report(y, y_hat))
    print('-'*40)
    print('Macierz pomyłek')
    print()
    print(confusion_matrix(y, y_hat))
    print('-'*40)

# Dane
Dane do trenowania modelu znajdują się w pliku  `final_train.csv` i zostały dostarczone przez organizatorów.

## Pochodzenie danych
Dane zawierają informacje o aktywnościach uczestników biorących udział w eksperymencie. 

Zbiór danych składa się z sześciu podstawowych czynności: trzech postaw statycznych (stanie, siedzenie, leżenie) oraz trzech czynności dynamicznych (chodzenie, schodzenie po schodach i wchodzenie po schodach). 

Eksperyment polegał na noszeniu przypiętego na pasku smartfona (Samsung Galaxy S II).

![experiment](./docs/img/experiment.png)

Urządzenie było przypięte na pasku tak jak przedstawiono na zdjęciu poniżej:

![smartphone](./docs/img/smartphone.PNG)

Szczególnie cenną informacją jest skierowanie osi wymiarów.

Przechwytywano 3-wymiarowe przyspieszenie liniowe i 3-wymiarową prędkość kątową w stałej częstotliwości 50Hz przy użyciu wbudowanego w urządzenie akcelerometru i żyroskopu.

## Wczytanie danych


In [3]:
df = pd.read_csv(DATA_PATH, index_col=0)
df

Aby lepiej zrozumieć dane skupmy się na przetłumaczeniu kilku nazw:

### Aktywności:
* Laying - Leżenie
* Sitting - Siedzenie
* Standing - Stanie
* Walking - Chodzenie
* Walking upstairs - Chodzenie po schodach do góry
* Walking downstairs - Chodzenie po schodach w dół

Jeśli chcemy tworzyć klasyfikator aktywności, warto sprawdzić, czy klasy sa zbalansowane. W tym celu zliczamy wystąpienia poszczególnych klas w naszym zestawie danych.

In [5]:
df['Activity'].value_counts()

LAYING                2167
SITTING               2019
STANDING              1979
WALKING               1900
WALKING_UPSTAIRS      1633
WALKING_DOWNSTAIRS    1388
Name: Activity, dtype: int64

W rozkładzie klas nie występuje problem niezbalansowania.

Nazwy kolumn posiadają przedrostki, oznaczają one poszczególne agregacje jakich dokonano na sygnałach z danego okna.
Znaczenie nazw przedrostków:

| Function | Description |
|----------|-------------|
| mean     | Mean value  |
| std      | Standard deviation   |
| mad      | Median absolute value             |
| max      | Largest values in array           |
| min      | Smallest value in array           |
| sma      | Signal magnitude area             |
| energy   | Average sum of the squares        |
| iqr      | Interquartile range               |
| entropy  | Signal Entropy                    |
| arCoeff  | Autorregresion coefficients       |
| correlation | Correlation coefficient        |
| maxFreqInd | Largest frequency component     |
| meanFreq | Frequency signal weighted average |
| skewness | Frequency signal Skewness         |
| kurtosis | Frequency signal Kurtosis         |
| energyBand | Energy of a frequency interval  |
| angle    | Angle between two vectors         |

Poszczególne kolumny to skrócone nazwy atrybutów takich jak:
* Gyro (z ang. *gyroscope*) - żyroskop
* Acc (z ang. *accelerometer*) - akcelerometr
* Mag (z ang. *magnitude*) - wielkość
* Jerk - zmiana przyspieszenia/zryw


# Eksplorcja danych
Dla każdej każdej zmiennej w każdej z klas aktywności przeprowdzamy analizę statystyczną.

Wyniki znajdują się w katalogu `./reports/eda`.

Analizy posłużyły nam do zapoznania się ze zbiorem danych, wartościami przyjmowanymi przez poszczególne zmienne, a także do interpretencji stworzonych modeli.

Podczas analizy zauważyliśmy, że w każdej ze zmiennych są dwie bardzo często występujące wartości, które zaburzają rozkłady zmiennych. Możemy wnioskować, że są to błędne pomiary.

Interesujący jest fakt, że te obserwacje dotyczą tylko dwóch klas: leżenie i wchodzenie po schodach.


# Model 1 
## Zasada działania

* Przetworzenie danych
    * Usunięcie zmiennych `subject` i `void()`
* Balansowanie
    * Nie wykorzystano balansowania zmiennych
* Transformacje zmiennych
    * Zamiana zmiennej `Activity` według podziału na 3 grupy:
        * `WALKING_ALL`: `WALKING`, `WALKING_DOWNSTAIRS`, `WALKING_UPSTAIRS`
        * `STANDING_SITTING`: `STANDING`, `SITTING`
        * `LAYING`: `LAYING`
* Klasyfikator
    * Drzewo klasyfikacyjne

Dokładny opis modelu wraz z analizą wyników znajduje się w notatniku `Model-3CategoriesTree-model`.


## Zalety
 Dzięki temu, że model korzysta tylko z dwóch zmiennych może być stosowany w urządzeniach z ograniczoną pamięcią, ponieważ wystarczy zbierać tylko dwie zmienne. Co więcej obie z nich są rejestrowane przez akcelerometr, więc możemy pominąć używanie żyroskopu. Napewno pozytywnie to wpłynie również na zużycie baterii.

Implementację możemy sprowadzić do dwóch wyrażeń warunkowych ` if `, dzięki temu nie będzie problemu z osadzeniem modelu w różnych urządzeniach.

## Zastosowania
Model może zostać użyty w przypadku, gdy chcemy rozróżnić ogólną dynamikę zachowania człowieka.

Ze względu na 100% skuteczność, modelu możemy używać w sytuacjach, gdy od niezawodnosci modelu zależy np. życie człowieka.

 

# Model 2
## Możliwe zastosowania:
 Dzięki temu, że model korzysta tylko z dwóch zmiennych może być stosowany w urządzeniach z ograniczoną pamięcią, a jego implementację możemy sprowadzić do dwóch wyrażeń warunkowych ` if `.

# Model 3

In [7]:
x, y = df.drop(columns = ['Activity']), df['Activity']

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=RANDOM_STATE)

In [ ]:
model_automl = joblib.load(MODEL_AUTOML_PATH)
model_decision_tree = joblib.load(MODEL_DECISION_TREE_PATH)
model_decision_tree_3_cat = joblib.load(MODEL_DECISION_TREE_3_CAT_PATH)

# AutoML model - SVM

Steps:

* Data preprocessing
    * Imputation strategy: Median
    * Rescaling: Min Max
* Balancing
    * No class balancing was used
* Feature preprocessor
    * No additional feature preprocessing
* Classifier
    * SVM with RBF kernel

In [9]:
summary(model_automl, x_train, y_train)

----------------------------------------
Raport klasyfikacyjny

                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00      1650
           SITTING       0.99      0.99      0.99      1479
          STANDING       1.00      0.99      0.99      1500
           WALKING       1.00      1.00      1.00      1417
WALKING_DOWNSTAIRS       1.00      1.00      1.00      1047
  WALKING_UPSTAIRS       1.00      1.00      1.00      1221

          accuracy                           1.00      8314
         macro avg       1.00      1.00      1.00      8314
      weighted avg       1.00      1.00      1.00      8314

----------------------------------------
Macierz pomyłek

[[1650    0    0    0    0    0]
 [   0 1471    7    0    0    1]
 [   0   17 1483    0    0    0]
 [   0    0    0 1417    0    0]
 [   0    0    0    0 1047    0]
 [   0    0    0    0    0 1221]]
----------------------------------------


In [10]:
summary(model_automl, x_test, y_test)

----------------------------------------
Raport klasyfikacyjny

                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       517
           SITTING       0.98      0.97      0.97       540
          STANDING       0.96      0.97      0.97       479
           WALKING       1.00      1.00      1.00       483
WALKING_DOWNSTAIRS       0.99      1.00      0.99       341
  WALKING_UPSTAIRS       1.00      0.99      1.00       412

          accuracy                           0.99      2772
         macro avg       0.99      0.99      0.99      2772
      weighted avg       0.99      0.99      0.99      2772

----------------------------------------
Macierz pomyłek

[[517   0   0   0   0   0]
 [  0 523  17   0   0   0]
 [  0  13 466   0   0   0]
 [  0   0   0 483   0   0]
 [  0   0   0   1 340   0]
 [  0   0   0   0   4 408]]
----------------------------------------
